In [ ]:
library(kBET)
library(Matrix)
library(useful)
library(reshape)
library(dplyr)

In [ ]:
#Import UMAP coordinates derived from Harmony-adjusted PCs and run kBET for each cluster

setwd("/home/ngr18/covid/k_bet")

umap<-read.csv('umap_coords_harmony.csv', row.names = 1)

clusters<-read.csv('clustering_harmony.csv', row.names = 1)

kBET.umi.counts <- list()
for (cluster in unique(clusters$initial_clustering)){
    keep<-rownames(clusters[clusters$initial_clustering == cluster,])
    kBET.umi.counts[[cluster]] <- kBET(df = umap[rownames(umap) %in% keep,], 
                                     batch = clusters[rownames(clusters) %in% keep,]$sample_id,
                                     plot = FALSE)
}

df <- data.frame(score = c(0,0,0))
for (cluster in unique(clusters$initial_clustering)){
    label<-paste0(cluster, '_score')
    df$label<-kBET.umi.counts$cluster[['summary']][2:4,2]
}

saveRDS(kBET.umi.counts, "kbet_result_harmony.RDS")

In [ ]:
#Import UMAP coordinates without any batch correction and run kBET for each cluster

setwd("/home/ngr18/covid/k_bet")

umap<-read.csv('umap_coords_harmony.csv', row.names = 1)

clusters<-read.csv('clustering_harmony.csv', row.names = 1)

kBET.umi.counts <- list()
for (cluster in unique(clusters$initial_clustering)){
    keep<-rownames(clusters[clusters$initial_clustering == cluster,])
    kBET.umi.counts[[cluster]] <- kBET(df = umap[rownames(umap) %in% keep,], 
                                     batch = clusters[rownames(clusters) %in% keep,]$sample_id,
                                     plot = FALSE)
}

df <- data.frame(score = c(0,0,0))
for (cluster in unique(clusters$initial_clustering)){
    label<-paste0(cluster, '_score')
    df$label<-kBET.umi.counts$cluster[['summary']][2:4,2]
}

saveRDS(kBET.umi.counts, "kbet_result_harmony.RDS")

In [ ]:
#Above steps run on cluster, so reimport results

harmony<-readRDS("kbet_result_harmony.RDS")
no_harmony<-readRDS("kbet_result_no_harmony.RDS")

#remove empty result
cluster_names<-unique(names(harmony))
cluster_names<-cluster_names[-18]

#combine kBet output in to single dataframe (harmony)
harmony_result<-data.frame(scores = c(0,0,0))
for (i in 1:length(cluster_names)){
    temp<-harmony[[cluster_names[i]]][['summary']][2:4,2]
   
    harmony_result<-cbind(harmony_result, temp)
    colnames(harmony_result)[i+1]<-cluster_names[i]
}

harmony_result$scores<-NULL
a<-melt(harmony_result)
a$treatment<-"harmony"

#combine kBet output in to single dataframe (no harmony)

no_harmony_result<-data.frame(scores = c(0,0,0))
for (i in 1:length(cluster_names)){
    temp<-no_harmony[[cluster_names[i]]][['summary']][2:4,2]
   
    no_harmony_result<-cbind(no_harmony_result, temp)
    colnames(no_harmony_result)[i+1]<-cluster_names[i]
}

no_harmony_result$scores<-NULL
b<-melt(no_harmony_result)
b$treatment<-"no_harmony"

#combine both
d<-rbind(a,b)

#plot
options(repr.plot.width=7, repr.plot.height=5)

ggplot(d, aes(variable, value, color = treatment)) +
geom_boxplot(varwidth=T) + theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
panel.background = element_blank(), axis.line = element_line(colour = "black"))+ ylim(0, 1)+
theme(axis.title.x = element_blank(),
  axis.title.y = element_blank())+
theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1, size = 20)) +
theme(axis.text.y = element_text(size = 20))


setwd("/home/ngr18/covid/figures")
ggsave("kbet_plot.pdf", width = 10, height = 10)